In [ ]:
import os, cv2, random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../data"]).decode("utf8"))

In [ ]:
TRAIN_1_DIR = '../data/train/1/'
TRAIN_0_DIR = '../data/train/0/'


ROWS = 36
COLS = 36
CHANNELS = 1

train_1 =   [TRAIN_1_DIR+i for i in os.listdir(TRAIN_1_DIR)]
train_0 =   [TRAIN_0_DIR+i for i in os.listdir(TRAIN_0_DIR)]

train_images = train_1[:26000] + train_0[:26000]

print(len(train_images))
print(len(train_1))
print(len(train_0))

In [ ]:
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) #
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)


def prep_data(images):
    count = len(images)
    data = np.ndarray((count, ROWS, COLS), dtype=np.uint8)

    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image.T
        if i%250 == 0: print('Processed {} of {}'.format(i, count))
    
    return data


train = prep_data(train_images)
train_and_label = zip(train, ([1]*len(train_1)) + ([0]*len(train_0)))

print("Train shape: {}".format(train.shape))

In [ ]:
def show_example(idx):
    face = read_image(train_1[idx])
    noface = read_image(train_0[idx])
    pair = np.concatenate((face, noface), axis=1)
    plt.figure(figsize=(10,5))
    plt.imshow(pair, cmap='gray')
    plt.show()
    

show_example(1)

In [ ]:
show_example(2)

In [ ]:
show_example(4)

In [ ]:
train_and_label = list(zip(train, ([1]*26000 + ([0]*26000))))

In [ ]:
train_and_label[26000]

In [ ]:
random.shuffle(train_and_label)

In [ ]:
list(map(lambda x: x[1], train_and_label))[:20]

In [ ]:
images = np.array(list(map(lambda x: x[0], train_and_label)))
images.resize((52000, 36, 36, 1))
labels = np.array(list(map(lambda x: x[1], train_and_label)))

In [ ]:
optimizer = RMSprop(lr=1e-4)
objective = 'binary_crossentropy'


def faceRecognition():
    
    model = Sequential()

    model.add(Conv2D(32, 3, strides=(1,1), border_mode='same',
                     input_shape=(36, 36, 1), data_format="channels_last", activation='relu'))
    model.add(Conv2D(32, 3, strides=(1,1), border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, 3, strides=(1,1), border_mode='same', activation='relu'))
    model.add(Conv2D(64, 3, strides=(1,1), border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, 3, strides=(1,1), border_mode='same', activation='relu'))
    model.add(Conv2D(128, 3, strides=(1,1), border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(256, 3, strides=(1,1), border_mode='same', activation='relu'))
    model.add(Conv2D(256, 3, strides=(1,1), border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
    return model


model = faceRecognition()

In [ ]:
model.fit(images, labels, batch_size=32, epochs=2)